# 这是我的第三个练习

动态规划(DP)可以处理的问题有什么特点 ？    
**最优解可以由该问题的子问题最优解求得**

简单描述动态规划的几个步骤？    
**确认子问题，初始化子问题，确定迭代公式，得到最优解**

## 完成编辑距离解码函数

In [260]:
solution = {}

In [261]:
from functools import lru_cache

In [262]:
@lru_cache(maxsize=2**10)
def edit_distance(string1, string2):
    '''找出string1与string2的最小编辑距离'''
    
    # 如果两个词有一个为空，则编辑距离就是另外一个词的长度
    if len(string1) == 0: return len(string2)
    if len(string2) == 0: return len(string1)
    
    tail_s1 = string1[-1]
    tail_s2 = string2[-1]
    
    # 求删除的string1的尾部与string2的编辑距离
    # 求添加string2的尾部至string1的尾部与string2的编辑距离
    candidates = [
        (edit_distance(string1[:-1], string2) + 1, 'DEL {}'.format(tail_s1)),  
        # string 1 delete tail
        (edit_distance(string1, string2[:-1]) + 1, 'ADD {}'.format(tail_s2)),  
        # string 1 add tail of string2
    ]
    
    # 如果两个尾部一样，说明无需替换尾部，直接计算string1和string2都减去尾部的编辑距离即可
    if tail_s1 == tail_s2:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 0, '')
    # 如果两个尾部不一样，说明需要替换string1的尾部成string2的尾部，编辑距离加1
    else:
        both_forward = (edit_distance(string1[:-1], string2[:-1]) + 1, 'SUB {} => {}'.format(tail_s1, tail_s2))

    candidates.append(both_forward)
    
    # 比较三种可能的操作下，最小的编辑距离和操作
    min_distance, operation = min(candidates, key=lambda x: x[0])
    
    solution[(string1, string2)] = operation 
    
    return min_distance

## 拼音自动纠错

In [269]:
from collections import Counter
import re

def token(string):
    '''转换成拼音'''
    return pinyin.get(string,format='strip',delimiter=' ')

def tokens(text):
    "List all the pinyin characters"
    return re.findall('[a-z]+',text.lower())

In [270]:
import pinyin

In [271]:
chinese_dataset = "/Users/duxy/Downloads/Study/NLP/NLP技能班/dateset/article_9k.txt"
all_string = open(chinese_dataset).read()
len(all_string)

33425826

In [272]:
all_pinyin = token(all_string)
len(all_pinyin)

129433034

In [273]:
PINYIN_COUNT = Counter(tokens(all_pinyin))

In [274]:
gram_1 = []
gram_2 = []
for i, line in enumerate(open(chinese_dataset)):
    if i % 10000 == 0:
        print(i)
    line_pinyin = token(line)
    seg_pinnyin = line_pinyin.split()
    gram_1 += seg_pinnyin
    gram_2 += [seg_pinnyin[index]+" "+seg_pinnyin[index+1] for index in range(len(seg_pinnyin)-1)]
print(f"1-gram {len(gram_1)}")
print(f"2-gram {len(gram_2)}")
    
gram1_freq = Counter(gram_1)
gram2_freq = Counter(gram_2)

0
10000
20000
30000
40000
50000
60000
70000
80000
1-gram 33336215
2-gram 33246606


In [275]:
def pro_1(freq, pinyin):
    return freq[pinyin]/len(gram_1) if pinyin in freq else 1/len(gram_1)

def pro_2(freq, py1, py2):
    if py2 == "":
        return pro_1(gram1_freq, py1)
    elif py1 == "":
        return pro_1(gram1_freq, py2)
    return freq[py1+" "+py2]/len(gram_2) if py1+" "+py2 in gram2_freq else 1/len(gram_2)

def get_sentence_prob(sentence_py):
    sentence_py = sentence_py.split()
#     print(sentence_py)
    prob = 1
    for i in range(len(sentence_py)-1):
        prob *= pro_2(gram2_freq, sentence_py[i], sentence_py[i+1])
    return prob

In [276]:
print(f"wi: {pro_1(gram1_freq, 'wi')}")
print(f"wo: {pro_1(gram1_freq, 'wo')}")
print(f"xiao mi: {pro_2(gram2_freq, 'xiao', 'mi')}")
print(f"xi aomi: {pro_2(gram2_freq, 'xi', 'aomi')}")
print(f"xiao mi shou ji: {get_sentence_prob('xiao mi shou ji')}")
print(f"xia mi shou ji: {get_sentence_prob('xia mi shou ji')}")
get_sentence_prob("z")

wi: 2.9997406724188696e-08
wo: 0.001939662316192765
xiao mi: 1.1249268571955886e-05
xi aomi: 3.007825821378579e-08
xiao mi shou ji: 4.0288402672042684e-15
xia mi shou ji: 2.585352043125734e-16


1

In [295]:

from functools import lru_cache

@lru_cache(maxsize=2**10)
def split_pinyin(string):
    if len(string) == 1:
        return string, pro_1(gram1_freq,string)
    if len(string)<=5:
        result = sorted([(string[:i],string[i:],pro_2(gram2_freq, string[:i], string[i:])) for i in range(len(string)-1)])
        best_result = result[0]
        if best_result[1]==string:
            return best_result[1], best_result[2]
        string1, prob1 = split_pinyin(best_result[0])
        string2, prob2 = split_pinyin(best_result[1])
        
        return string1 + " " + string2, prob1*prob2
    else:
        candidate = []
        for i in range(1,5):
            string1, prob1 = split_pinyin(string[:len(string)-5+i])
            string2, prob2 = split_pinyin(string[len(string)-5+i:])
            candidate.append((string1+" "+string2, prob1*prob2))
        candidate = sorted(candidate, key=lambda x:x[1])
        best_candidate = candidate[-1]

        return best_candidate[0], best_candidate[1]
        
    
    

    

In [296]:
!time
split_pinyin("xiaomishouji")


real	0m0.001s
user	0m0.000s
sys	0m0.000s


('xiao mi shou ji', 4.751797169058313e-10)

In [297]:
split_pinyin("zheshiyitiaoshenqidetianlu")

('zhe shi yi tiao shen qi de tian lu', 6.430052806729468e-21)

In [298]:
split_pinyin("huaweiniubi")

('hua wei niu bi', 1.428563549285516e-10)

In [299]:
alphabet = 'abcdefghijklmnopqrstuvwxyz'

def splits(word):
    'Return a list of all possible (first, rest) pairs that comprise pinyin.'
    return [(word[:i], word[i:])
           for i in range(len(word)+1)]

def edits1(word):
    'Return all strings that are one edit away from this pinyin.'
    pairs = splits(word)   
    deletes = [a+b[1:] for (a,b) in pairs if b] # input pinyin -删除操作 去掉b[0]
    replaces = [a+c+b[1:] for (a,b) in pairs for c in alphabet if b] #替换  b[0]替换为c
    inserts = [a+c+b for (a,b) in pairs for c in alphabet] #插入操作 在a和b之间插入字母
    return set(deletes + replaces + inserts)

def known(words):
    'Return the pinyin in our data'
    return {w for w in words if w in PINYIN_COUNT}

def edits0(word):
    'Return all strings that are zero edits away from word (i.e., just word itself).'
    return {word}

def edits2(word):
    'Return all strings that are two edits away from this pinyin.'
    return {e2 for e1 in edits1(word) for e2 in edits1(e1)} #要计算编辑距离为2的词 可以通过计算与原词编辑距离为1的词的编辑距离为1的词

In [300]:
def correct(word): #pinyin candidates piyin pingyin aogyin .... 
    'Find the most possible pinyin based on edit distance'
    # Prefer edit distance 0, then 1, then 2; otherwist default to word itself
    candidates = (known(edits0(word)) or
                  known(edits1(word)) or
                  known(edits2(word)) or
                  [word])  #计算所有与输入词编辑距离为0，1，2的词
    return max(candidates,key=PINYIN_COUNT.get) #取出在所有candidate里在语料库里出现次数最多的词

In [320]:
def parse_results(string):
    pinyin = split_pinyin(string)[0]
    pinyin = pinyin.split()
    pinyin = map(correct, pinyin)
    return (" ").join(pinyin)
parse_results("xinhuashebbodao")        

'xin hua she b bo dao'